# Connect Comprehensive Stack Architecture

This document provides a detailed architectural overview of the **Connect Comprehensive Stack**, a production-ready Amazon Connect solution featuring Bedrock-primary conversational AI, intelligent tool calling with FastMCP 2.0, real-time hallucination detection, seamless agent handover, and a scalable **Data Lake** for advanced analytics.

## 1. System Context Diagram (Level 1)

This high-level view shows how the **Connect Comprehensive System** interacts with users and external banking systems.

```ascii
                                            +----------------------------------+
                                            |   Core Banking Systems           |
                                            |   (Accounts, Cards, Transactions)|
                                            +----------------^+----------------+
                                                             |
                                                             | (Secure API / VPC Link)
                                                             |
    +--------------+              +--------------------------+-----------------------+
    |              |              |                                                  |
    |   Customer   +------------->+      Connect Comprehensive System            |
    |  (Voice/Chat)|    HTTPS/    |      (Amazon Connect + Federated Bots)           |
    |              |    SIP       |                                                  |
    +--------------+              +--------------------------+-----------------------+
                                                             |
                                                             | (Analytics & Insights)
                                                             v
                                            +----------------------------------+
                                            |        Data Lake & BI            |
                                            |  (Athena / QuickSight / Tableau) |
                                            +----------------------------------+
```

## 2. Container Diagram (Level 2)

This diagram details the *containers* (deployable units) within the system boundary. It illustrates the **Federated Hybrid Architecture**.

```ascii
    +-------------+         +----------------+
    | Customer    +-------->+ Amazon Connect |
    | Device      |         | Instance       |
    +-------------+         +---+------+-----+
                                |      |
          (Voice / Chat Entry)  |      |  (Contact Flows & Routing)
                                v      v
                    +-----------+------+------------------------+
                    |           Gateway Routing Layer           |
                    | (Amazon Lex V2 - Gateway Bot)             |
                    +-------------------+-----------------------+
                                        |
                  +---------------------+---------------------+
                  |                                           |
    +-------------v-----------+                 +-------------v-----------+
    |   Conversational AI     |                 |   Transactional Logic   |
    |   (Generative Path)     |                 |   (Specialized Path)    |
    |                         |                 |                         |
    | +---------------------+ |                 | +---------------------+ |
    | | Bedrock MCP Lambda  | |                 | | Specialized Bot(s)  | |
    | | (Orchestrator)      | |                 | | (Banking / Sales)   | |
    | +----------+----------+ |                 | +----------+----------+ |
    |            |            |                 |            |            |
    | +----------v----------+ |                 | +----------v----------+ |
    | | Claude 3.5 Sonnet   | |                 | | Specialized Lambda  | |
    | | (Reasoning & Tools) | |                 | | (Python Logic)      | |
    | +---------------------+ |                 | +----------+----------+ |
    +-------------------------+                 +------------+------------+
                                                             |
                                                             v
                                                +------------+------------+
                                                |   External Banking API  |
                                                +-------------------------+
```

## 3. High-Level Architecture (Federated & Hybrid)

The solution leverages a **Federated Hybrid Pattern** combining the flexibility of Bedrock-based Generative AI with the control and speed of specialized, domain-specific Bots.

*   **Gateway Bot (Router)**: The entry point (bedrock-primary) that uses Claude 3.5 Sonnet to understand intent and context.
*   **Specialized Bots**: Dedicated Lex V2 bots (Banking, Sales) that handle deterministic, high-compliance workflows.
*   **Connect Orchestrator**: The Contact Flow manages the routing between the Gateway and Specialized bots based on intent signals.

### Federated Architecture Sequence

```mermaid
sequenceDiagram
    participant User
    participant Connect as Amazon Connect
    participant Gateway as Gateway Bot (Lex)
    participant Bedrock as Bedrock MCP Lambda
    participant Specialized as Specialized Bot (Lex)
    participant AppLambda as Specialized Lambda

    User->>Connect: "I want to open an account"
    Connect->>Gateway: Invoke Gateway Bot
    Gateway->>Bedrock: Fallback Intent (Input Text)
    Bedrock->>Bedrock: Analyze Intent (Claude 3.5)
    Bedrock-->>Gateway: Response: [Intent: OpenAccount]
    Gateway-->>Connect: Fulfilled (Intent Signal)
    Connect->>Connect: Check Intent Condition
    Connect->>Specialized: Transfer to Sales Bot
    Specialized->>AppLambda: "Open Account"
    AppLambda-->>Specialized: "What is your full name?"
    Specialized-->>Connect: Elicit Slot
    Connect-->>User: "What is your full name?"
```

## 4. Observability & Data Lake (Level 3 - Component)

A centralized Data Lake aggregates logs from all federated components, ensuring a unified view of the entire customer journey regardless of which bot handled the interaction.

### Unified Logging Pipeline Diagram

```ascii
      +---------------------+      +----------------------+      +----------------------+
      |  Gateway Bot        |      |  Banking Bot         |      |  Sales Bot           |
      |  (Conversation Logs)|      |  (Conversation Logs) |      |  (Conversation Logs) |
      +----------+----------+      +-----------+----------+      +----------+-----------+
                 |                             |                            |
                 v                             v                            v
      +----------+-----------------------------+----------------------------+-----------+
      |  CloudWatch Log Groups                                                          |
      |  (/aws/lex/gateway, /aws/lex/banking, /aws/lex/sales, /aws/lambda/*)             |
      +----------+----------------------------------------------------------------------+
                 |
                 | (Subscription Filters -> Kinesis Destination)
                 v
      +----------+-----------+        +----------------------+
      |  Kinesis Firehose    +------->| S3 Data Lake (Raw)   |
      |  (Log Aggregator)    |        | /cloudwatch-logs/    |
      +----------------------+        +-----------+----------+
                                                  |
                                                  v
                                      +-----------+----------+
                                      | AWS Glue Crawler     |
                                      +-----------+----------+
                                                  |
                                                  v
                                      +-----------+----------+
                                      | Amazon Athena        |
                                      | (Standard SQL)       |
                                      +----------------------+
```

### Data Latency Sequence

```ascii
      +----------------+       +----------------+       +----------------+       +----------------+
      | Amazon Connect |       |   EventBridge  |       | Kinesis        |       | Data Lake      |
      | Instance       |       |   Rule         |       | Firehose       |       | (S3/Athena)    |
      +-------+--------+       +-------+--------+       +-------+--------+       +-------+--------+
              |                        |                        |                        |
              | (Contact State Change) |                        |                        |
              +----------------------->| (Match Rule)           |                        |
              |                        +----------------------->|                        |
              |                        |                        +----------------------->| (Buffer & Write)
              |                        |                        |                        |
              | (Queued)               |                        |                        |
              +----------------------->|                        |                        |
              |                        |                        +----------------------->| (Row: Status=Queued)
              v                        v                        v                        v
```

## 5. Metrics Source Mapping

The following table details which analytical metrics are derived from which source component in the Data Lake.

| Metric Component | Source Pipeline | Primary Table | Key Metrics Derived | Latency |
| :--- | :--- | :--- | :--- | :--- |
| **Contact Trace Records (CTR)** | Kinesis Stream -> Firehose | `ctrs` | IVR Containment, AHT, Disconnect Reasons, Journey Outcome, Transfer Rate | Near Real-time (~2-3 min) |
| **Agent Events** | Kinesis Stream -> Firehose | `agent_events` | Agent Availability, Login Duration, State Changes, Occupancy | Near Real-time |
| **Lifecycle Events** | EventBridge -> Firehose | `lifecycle_events` | **Live Queue Backlog**, Point-in-time Contact States, Raw Event History | Real-time (<60s) |
| **Federated Bot Logs** | CloudWatch -> Firehose | `cloudwatch_logs` | **Intent Routing Accuracy**, Bot Latency, Error Rates | ~5 mins |
| **AI Insights** | Lambda -> Kinesis -> Firehose | `ai_insights` | Hallucination Rate, Validation Latency, Jailbreak Attempts, Model Performance | Real-time |
| **Contact Lens** | S3 Export -> Glue Crawler | `contact_lens_analysis` | Sentiment Score, Interruption Rate, Non-Talk Time, Category Matches | Post-Call (5-10 min) |
| **System Health** | CloudWatch Metric Stream | Defined by Namespace | Throttling, Concurrent Calls, System Errors, API Usage | Real-time |

---

## 6. Component Deep Dive

### 6.1 Amazon Connect (The Orchestrator)
*   **Role**: Entry point and central router for all voice and chat interactions.
*   **Key Features**:
    *   **Gateway Contact Flow**: Routes to Gateway Bot initially.
    *   **Federated Routing**: Checks Gateway output signal to switch Flow execution to **Banking Bot** or **Sales Bot**.
    *   **Customer Queue Flow**: Manages wait experience with position updates and callback options
    *   **Contact Lens**: Real-time sentiment analysis and transcription
    *   **Queues**: GeneralAgentQueue for agent handover
    *   **Storage**: S3 storage for chat transcripts, call recordings, and contact trace records

### 6.2 Amazon Lex V2 (Federated Mesh)
*   **Gateway Bot**:
    *   **Role**: Smart Router. Uses Bedrock/LLM to categorize intent.
    *   **Fulfillment**: Bedrock MCP Lambda.
*   **Banking Bot**:
    *   **Role**: Specialized execution for "Check Balance", "Transfer".
    *   **Fulfillment**: Banking Lambda (Python).
*   **Sales Bot**:
    *   **Role**: Specialized execution for "Open Account".
    *   **Fulfillment**: Sales Lambda (Python).

### 6.3 Logging & Data Lake
*   **Unified Prefix**: All components write to `cloudwatch-logs/`.
*   **No Code Changes Required for Analytics**: Queries written for the monolithic bot work for the federated bots because the log structure (JSON) and storage path remain consistent.
